# AutoEncoders

## Download the Dataset

### ML-100k

In [1]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2022-03-30 13:22:59--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  5.09MB/s    in 0.9s    

2022-03-30 13:23:00 (5.09 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

### ML-1M

In [2]:
!wget "http://files.grouplens.org/dataset/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2022-03-30 13:28:30--  http://files.grouplens.org/dataset/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-03-30 13:28:31 ERROR 404: Not Found.

unzip:  cannot find or open ml-1m.zip, ml-1m.zip.zip or ml-1m.zip.ZIP.
ml-100k  ml-100k.zip  sample_data


### Importing the Libraries

In [21]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

### Importing the Dataset

### Preparing the training set and the test set

In [22]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

Getting the number of users and movies

In [23]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

### Converting the data into an array with users in lines and movies in columns

In [24]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

### Converting the data into Torch tensors

In [25]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

### Creating the architecture of the Neural Network

In [26]:
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 20)
    self.fc2 = nn.Linear(20, 10)
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20, nb_movies)
    self.activation = nn.Sigmoid()
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay=0.5)

### Training the SAE

In [27]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7715)
epoch: 2loss: tensor(1.0967)
epoch: 3loss: tensor(1.0532)
epoch: 4loss: tensor(1.0384)
epoch: 5loss: tensor(1.0311)
epoch: 6loss: tensor(1.0267)
epoch: 7loss: tensor(1.0237)
epoch: 8loss: tensor(1.0220)
epoch: 9loss: tensor(1.0209)
epoch: 10loss: tensor(1.0195)
epoch: 11loss: tensor(1.0190)
epoch: 12loss: tensor(1.0184)
epoch: 13loss: tensor(1.0180)
epoch: 14loss: tensor(1.0173)
epoch: 15loss: tensor(1.0173)
epoch: 16loss: tensor(1.0170)
epoch: 17loss: tensor(1.0171)
epoch: 18loss: tensor(1.0164)
epoch: 19loss: tensor(1.0162)
epoch: 20loss: tensor(1.0160)
epoch: 21loss: tensor(1.0162)
epoch: 22loss: tensor(1.0158)
epoch: 23loss: tensor(1.0160)
epoch: 24loss: tensor(1.0158)
epoch: 25loss: tensor(1.0156)
epoch: 26loss: tensor(1.0155)
epoch: 27loss: tensor(1.0155)
epoch: 28loss: tensor(1.0149)
epoch: 29loss: tensor(1.0137)
epoch: 30loss: tensor(1.0123)
epoch: 31loss: tensor(1.0097)
epoch: 32loss: tensor(1.0086)
epoch: 33loss: tensor(1.0049)
epoch: 34loss: tens

### Testing the SAE

In [28]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9524)
